In [71]:
import sklearn
import pandas as pd 
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from functools import partial
from pathlib import Path
import os

In [72]:
df_covid = pd.read_csv("../../data_processed/usa_top30_with_density_smoothed.csv")
df_covid['date'] = pd.to_datetime(df_covid['date'], infer_datetime_format=True)

In [73]:
df_cams = pd.read_csv("../../data/cams_data.csv")
df_cams['time'] = pd.to_datetime(df_cams['time'], infer_datetime_format=True)
df_cams['time'] = df_cams['time'].apply(lambda x : x.date())

In [74]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39109 entries, 0 to 39108
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   shapeID                         39109 non-null  int64         
 1   GEOID                           39109 non-null  int64         
 2   NAME                            39109 non-null  object        
 3   date                            39109 non-null  datetime64[ns]
 4   cumulative_confirmed            39109 non-null  float64       
 5   smoothed_cumul                  39109 non-null  float64       
 6   smoothed_d1                     39109 non-null  float64       
 7   smoothed_d2                     39109 non-null  float64       
 8   mobility_transit_stations       39109 non-null  float64       
 9   mobility_retail_and_recreation  39109 non-null  float64       
 10  mobility_grocery_and_pharmacy   39109 non-null  float64       
 11  mo

In [75]:
df_covid

,shapeID,GEOID,NAME,date,cumulative_confirmed,smoothed_cumul,smoothed_d1,smoothed_d2,mobility_transit_stations,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_residential,mobility_workplaces
0,13,17197,Will,2020-03-25,50.0,44.095238,21.714286,9.809524,-28.000000,-42.000000,-13.000000,37.000000,22.000000,-49.000000
1,13,17197,Will,2020-03-26,67.0,71.476190,29.357143,8.333333,-33.000000,-46.000000,-14.000000,-15.000000,23.000000,-50.000000
2,13,17197,Will,2020-03-27,104.0,109.333333,34.750000,3.690476,-32.000000,-47.000000,-15.000000,-16.000000,23.000000,-49.000000
3,13,17197,Will,2020-03-28,156.0,151.666667,33.428571,-2.904762,-28.000000,-52.000000,-18.000000,-24.000000,16.000000,-34.000000
4,13,17197,Will,2020-03-29,192.0,182.952381,38.607143,0.880952,-28.000000,-54.000000,-27.000000,-36.000000,14.000000,-36.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39104,3231,55079,Milwaukee,2020-10-30,44609.0,44724.190476,713.964286,18.547619,-26.000000,-18.000000,-7.000000,29.000000,11.000000,-32.000000
39105,3231,55079,Milwaukee,2020-10-31,45652.0,45480.714286,755.214286,1.000000,-24.000000,-20.000000,1.000000,91.000000,3.000000,-8.000000
39106,3231,55079,Milwaukee,2020-11-01,46281.0,46316.761905,686.107143,-91.309524,-34.000000,-22.000000,-9.000000,2.000000,4.000000,-13.000000
39107,3231,55079,Milwaukee,2020-11-02,46835.0,47045.380952,552.142857,-178.904762,-33.113208,-21.509434,-8.716981,1.852941,4.018182,-12.773585


In [76]:
df_cams = df_cams.groupby(['time','shapeID']).agg([np.mean,np.min,np.max])
df_cams

x0                  x1                  x2  \
                        mean      amin      mean      amin      mean   
time       shapeID                                                     
2020-01-01 0        2.746214  1.318739 -0.468615 -1.177848 -0.196347   
           1        1.748343  1.136316 -1.344321 -2.492807  0.043200   
           2        2.537780  0.904072 -0.879046 -1.434459 -0.283907   
           3        2.232631  1.758709 -0.887527 -2.061887 -0.039456   
           4        1.863745  1.278517 -1.410770 -2.542341 -0.033701   
...                      ...       ...       ...       ...       ...   
2020-10-09 3228    -2.578400 -3.521142 -0.764019 -1.945059  0.338535   
           3229     1.005270 -1.254299 -0.073360 -1.027821 -0.528851   
           3230    -0.703218 -1.961407  1.287013 -0.388428  1.720019   
           3231    -1.562584 -4.393294 -1.495783 -2.678844 -0.311439   
           3232    -0.778073 -3.746977 -1.150497 -2.576713 -0.018446   

                                    x3                  x4            ...  \
                        amin      mean      amin      mean      amin  ...   
time       shapeID                                                    ...   
2020-01-01 0       -1.236057 -0.230875 -1.197603  0.771363 -0.113802  ...   
           1       -1.023753 -0.100307 -0.936233 -0.318099 -0.898874  ...   
           2       -1.005673 -0.232938 -1.075211  0.613301 -0.965509  ...   
           3       -0.979300 -0.188379 -1.040355  0.092777 -0.439938  ...   
           4       -1.106488 -0.133756 -1.003558 -0.190875 -0.768248  ...   
...                      ...       ...       ...       ...       ...  ...   
2020-10-09 3228    -0.778648  0.284701 -0.965865 -0.628849 -1.739375  ...   
           3229    -1.276024 -0.556423 -2.077708  0.115092 -0.395445  ...   
           3230    -0.771418  0.104234 -1.181206 -0.584574 -1.619687  ...   
           3231    -1.377934  0.256350 -1.153596  1.032231 -0.625120  ...   
           3232    -1.898842  0.501033 -0.495423  0.664446 -0.337451  ...   

                          x6                  x7                  x8  \
                        mean      amin      mean      amin      mean   
time       shapeID                                                     
2020-01-01 0        0.873012  0.603753  0.533494  0.277017 -0.245813   
           1        0.763758  0.361931  0.585046  0.434009 -0.598203   
           2        0.707564  0.438193  0.665952  0.509067 -0.396858   
           3        0.731009  0.379955  0.579249  0.497502 -0.457559   
           4        0.768530  0.336689  0.669830  0.593015 -0.647023   
...                      ...       ...       ...       ...       ...   
2020-10-09 3228     0.804540  0.035234  1.602055  1.297609 -1.888306   
           3229    -0.609240 -1.053243 -0.169344 -0.952326 -1.050392   
           3230    -1.052479 -1.614661 -0.032596 -0.217834 -2.009938   
           3231     0.278423 -0.533177  1.145559 -0.632924 -1.170794   
           3232     0.232678 -0.381604  0.762553 -1.120657 -1.018824   

                                    x9                 x10            
                        amin      mean      amin      mean      amin  
time       shapeID                                                    
2020-01-01 0       -0.744365  0.402408  0.005759  0.705195  0.001659  
           1       -0.794981  0.132206 -0.040984  0.150523 -0.042816  
           2       -0.731261  0.468917  0.106676  0.247497 -0.065500  
           3       -0.683913  0.202946  0.009591  0.349443  0.041207  
           4       -0.843513  0.175213  0.015534  0.073232 -0.174049  
...                      ...       ...       ...       ...       ...  
2020-10-09 3228    -2.382600  0.709347  0.209706  1.216620  0.623109  
           3229    -1.614056  0.607270 -0.264772  0.485292  0.285741  
           3230    -2.353993  0.183213 -0.378370  0.635360  0.159731  
           3231    -1.769246  1.366944  0.268053  1.285421  0.152326  
      

In [77]:
df_cams.columns = pd.Index([f"{x}_{y}" for (x,y) in df_cams.columns])
df_cams = df_cams.reset_index()

df_cams

,index,time_,shapeID_,x0_mean,x0_amin,x1_mean,x1_amin,x2_mean,x2_amin,x3_mean,...,x6_mean,x6_amin,x7_mean,x7_amin,x8_mean,x8_amin,x9_mean,x9_amin,x10_mean,x10_amin
0,0,2020-01-01,0,2.746214,1.318739,-0.468615,-1.177848,-0.196347,-1.236057,-0.230875,...,0.873012,0.603753,0.533494,0.277017,-0.245813,-0.744365,0.402408,0.005759,0.705195,0.001659
1,1,2020-01-01,1,1.748343,1.136316,-1.344321,-2.492807,0.043200,-1.023753,-0.100307,...,0.763758,0.361931,0.585046,0.434009,-0.598203,-0.794981,0.132206,-0.040984,0.150523,-0.042816
2,2,2020-01-01,2,2.537780,0.904072,-0.879046,-1.434459,-0.283907,-1.005673,-0.232938,...,0.707564,0.438193,0.665952,0.509067,-0.396858,-0.731261,0.468917,0.106676,0.247497,-0.065500
3,3,2020-01-01,3,2.232631,1.758709,-0.887527,-2.061887,-0.039456,-0.979300,-0.188379,...,0.731009,0.379955,0.579249,0.497502,-0.457559,-0.683913,0.202946,0.009591,0.349443,0.041207
4,4,2020-01-01,4,1.863745,1.278517,-1.410770,-2.542341,-0.033701,-1.106488,-0.133756,...,0.768530,0.336689,0.669830,0.593015,-0.647023,-0.843513,0.175213,0.015534,0.073232,-0.174049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914651,914651,2020-10-09,3228,-2.578400,-3.521142,-0.764019,-1.945059,0.338535,-0.778648,0.284701,...,0.804540,0.035234,1.602055,1.297609,-1.888306,-2.382600,0.709347,0.209706,1.216620,0.623109
914652,914652,2020-10-09,3229,1.005270,-1.254299,-0.073360,-1.027821,-0.528851,-1.276024,-0.556423,...,-0.609240,-1.053243,-0.169344,-0.952326,-1.050392,-1.614056,0.607270,-0.264772,0.485292,0.285741
914653,914653,2020-10-09,3230,-0.703218,-1.961407,1.287013,-0.388428,1.720019,-0.771418,0.104234,...,-1.052479,-1.614661,-0.032596,-0.217834,-2.009938,-2.353993,0.183213,-0.378370,0.635360,0.159731
914654,914654,2020-10-09,3231,-1.562584,-4.393294,-1.495783,-2.678844,-0.311439,-1.377934,0.256350,...,0.278423,-0.533177,1.145559,-0.632924,-1.170794,-1.769246,1.366944,0.268053,1.285421,0.152326


In [8]:
df_cams['time'] = pd.to_datetime(df_cams['time'], infer_datetime_format=True)

In [78]:
df_total = pd.merge(df_cams, df_covid, how="inner", left_on=['time','shapeID'], right_on=['date','shapeID'])
df_total.info()

KeyError: 'time'

In [67]:
df_covid[['date', 'shapeID']]

,date,shapeID
0,2020-03-25,13
1,2020-03-26,13
2,2020-03-27,13
3,2020-03-28,13
4,2020-03-29,13
...,...,...
39104,2020-10-30,3231
39105,2020-10-31,3231
39106,2020-11-01,3231
39107,2020-11-02,3231


In [80]:
df_cams

,index,time_,shapeID_,x0_mean,x0_amin,x1_mean,x1_amin,x2_mean,x2_amin,x3_mean,...,x6_mean,x6_amin,x7_mean,x7_amin,x8_mean,x8_amin,x9_mean,x9_amin,x10_mean,x10_amin
0,0,2020-01-01,0,2.746214,1.318739,-0.468615,-1.177848,-0.196347,-1.236057,-0.230875,...,0.873012,0.603753,0.533494,0.277017,-0.245813,-0.744365,0.402408,0.005759,0.705195,0.001659
1,1,2020-01-01,1,1.748343,1.136316,-1.344321,-2.492807,0.043200,-1.023753,-0.100307,...,0.763758,0.361931,0.585046,0.434009,-0.598203,-0.794981,0.132206,-0.040984,0.150523,-0.042816
2,2,2020-01-01,2,2.537780,0.904072,-0.879046,-1.434459,-0.283907,-1.005673,-0.232938,...,0.707564,0.438193,0.665952,0.509067,-0.396858,-0.731261,0.468917,0.106676,0.247497,-0.065500
3,3,2020-01-01,3,2.232631,1.758709,-0.887527,-2.061887,-0.039456,-0.979300,-0.188379,...,0.731009,0.379955,0.579249,0.497502,-0.457559,-0.683913,0.202946,0.009591,0.349443,0.041207
4,4,2020-01-01,4,1.863745,1.278517,-1.410770,-2.542341,-0.033701,-1.106488,-0.133756,...,0.768530,0.336689,0.669830,0.593015,-0.647023,-0.843513,0.175213,0.015534,0.073232,-0.174049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914651,914651,2020-10-09,3228,-2.578400,-3.521142,-0.764019,-1.945059,0.338535,-0.778648,0.284701,...,0.804540,0.035234,1.602055,1.297609,-1.888306,-2.382600,0.709347,0.209706,1.216620,0.623109
914652,914652,2020-10-09,3229,1.005270,-1.254299,-0.073360,-1.027821,-0.528851,-1.276024,-0.556423,...,-0.609240,-1.053243,-0.169344,-0.952326,-1.050392,-1.614056,0.607270,-0.264772,0.485292,0.285741
914653,914653,2020-10-09,3230,-0.703218,-1.961407,1.287013,-0.388428,1.720019,-0.771418,0.104234,...,-1.052479,-1.614661,-0.032596,-0.217834,-2.009938,-2.353993,0.183213,-0.378370,0.635360,0.159731
914654,914654,2020-10-09,3231,-1.562584,-4.393294,-1.495783,-2.678844,-0.311439,-1.377934,0.256350,...,0.278423,-0.533177,1.145559,-0.632924,-1.170794,-1.769246,1.366944,0.268053,1.285421,0.152326


In [57]:
df_cams

,time,shapeID,x0_mean,x0_amin,x1_mean,x1_amin,x2_mean,x2_amin,x3_mean,x3_amin,...,x6_mean,x6_amin,x7_mean,x7_amin,x8_mean,x8_amin,x9_mean,x9_amin,x10_mean,x10_amin
0,2020-01-01,0,2.746214,1.318739,-0.468615,-1.177848,-0.196347,-1.236057,-0.230875,-1.197603,...,0.873012,0.603753,0.533494,0.277017,-0.245813,-0.744365,0.402408,0.005759,0.705195,0.001659
1,2020-01-01,1,1.748343,1.136316,-1.344321,-2.492807,0.043200,-1.023753,-0.100307,-0.936233,...,0.763758,0.361931,0.585046,0.434009,-0.598203,-0.794981,0.132206,-0.040984,0.150523,-0.042816
2,2020-01-01,2,2.537780,0.904072,-0.879046,-1.434459,-0.283907,-1.005673,-0.232938,-1.075211,...,0.707564,0.438193,0.665952,0.509067,-0.396858,-0.731261,0.468917,0.106676,0.247497,-0.065500
3,2020-01-01,3,2.232631,1.758709,-0.887527,-2.061887,-0.039456,-0.979300,-0.188379,-1.040355,...,0.731009,0.379955,0.579249,0.497502,-0.457559,-0.683913,0.202946,0.009591,0.349443,0.041207
4,2020-01-01,4,1.863745,1.278517,-1.410770,-2.542341,-0.033701,-1.106488,-0.133756,-1.003558,...,0.768530,0.336689,0.669830,0.593015,-0.647023,-0.843513,0.175213,0.015534,0.073232,-0.174049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914651,2020-10-09,3228,-2.578400,-3.521142,-0.764019,-1.945059,0.338535,-0.778648,0.284701,-0.965865,...,0.804540,0.035234,1.602055,1.297609,-1.888306,-2.382600,0.709347,0.209706,1.216620,0.623109
914652,2020-10-09,3229,1.005270,-1.254299,-0.073360,-1.027821,-0.528851,-1.276024,-0.556423,-2.077708,...,-0.609240,-1.053243,-0.169344,-0.952326,-1.050392,-1.614056,0.607270,-0.264772,0.485292,0.285741
914653,2020-10-09,3230,-0.703218,-1.961407,1.287013,-0.388428,1.720019,-0.771418,0.104234,-1.181206,...,-1.052479,-1.614661,-0.032596,-0.217834,-2.009938,-2.353993,0.183213,-0.378370,0.635360,0.159731
914654,2020-10-09,3231,-1.562584,-4.393294,-1.495783,-2.678844,-0.311439,-1.377934,0.256350,-1.153596,...,0.278423,-0.533177,1.145559,-0.632924,-1.170794,-1.769246,1.366944,0.268053,1.285421,0.152326
